## 文本分类

### 从文本构建词向量

In [25]:
import numpy as np
import re
import random

In [4]:
# 创建一些实验样本
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],       #切分的词条
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    #类别标签向量，1代表侮辱性词汇，0代表不是
    classVec = [0,1,0,1,0,1]
    return postingList,classVec

In [5]:
# 创建词汇表
def createVocabList(dataSet):
    vocabSet = set([])                      
    for document in dataSet:
        #取并集
        vocabSet = vocabSet | set(document) 
    return list(vocabSet)

In [6]:
# 将inputSet向量化
def setOfWords2Vec(vocabList, inputSet):
    #创建一个其中所含元素都为0的向量
    returnVec = [0] * len(vocabList)      
    #遍历每个词条
    for word in inputSet:  
        #如果词条存在于词汇表中，则置1
        if word in vocabList:                                      
            returnVec[vocabList.index(word)] = 1
        else: print("the word: %s is not in my Vocabulary!" % word)
    return returnVec                                               

In [7]:
listOPosts,listClasses = loadDataSet()

In [13]:
myVocabList = createVocabList(listOPosts)
setOfWords2Vec(myVocabList,listOPosts[0])

[1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

### 从词向量训练概率

In [10]:
# 朴素贝叶斯训练函数
def trainNB0(trainMatrix,trainCategory):
    #计算训练的文档数目
    numTrainDocs = len(trainMatrix)  
     #计算每篇文档的词条数
    numWords = len(trainMatrix[0])     
    #文档属于侮辱类的概率
    pAbusive = sum(trainCategory)/float(numTrainDocs)   
    #创建numpy.zeros数组,词条出现数初始化为0
    p0Num = np.zeros(numWords); p1Num = np.zeros(numWords)
    #分母初始化为0
    p0Denom = 0.0; p1Denom = 0.0    
    for i in range(numTrainDocs):
        #统计属于侮辱类的条件概率所需的数据，即P(w0|1),P(w1|1),P(w2|1)···
        if trainCategory[i] == 1:   
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:                      
            #统计属于非侮辱类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)···
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = p1Num/p1Denom
    p0Vect = p0Num/p0Denom
    #返回属于侮辱类的条件概率数组，属于非侮辱类的条件概率数组，文档属于侮辱类的概率
    return p0Vect,p1Vect,pAbusive

In [11]:
# 朴素贝叶斯分类器分类函数
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    #对应元素相乘。logA * B = logA + logB，
    p1 = sum(vec2Classify * p1Vec) + np.log(pClass1)      
    #所以这里加上log(pClass1)
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

## 使用朴素贝叶斯过滤垃圾邮件

In [12]:
# 接收一个大字符串并将其解析为字符串列表
def textParse(bigString):    
    #将字符串转换为字符列表
    listOfTokens = re.split(r'\W*', bigString)                  
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]

In [46]:
# 测试朴素贝叶斯分类器
def spamTest():
    docList = []; classList = []; fullText = []
    #遍历25个txt文件
    try:
        for i in range(1, 26):                                             
            wordList = textParse(open('./email/spam/%d.txt' % i, 'r',encoding = 'UTF-8').read())
            docList.append(wordList)
            fullText.append(wordList)
            #标记垃圾邮件，1表示垃圾文件
            classList.append(1)                                            
            wordList = textParse(open('./email/ham/%d.txt' % i, 'r',encoding = 'UTF-8').read()) 
            docList.append(wordList)
            fullText.append(wordList)
            #标记非垃圾邮件，1表示垃圾文件
            classList.append(0) 
    except UnicodeDecodeError:
        print(i)
    #创建词汇表，不重复
    vocabList = createVocabList(docList)       
    #创建存储训练集的索引值的列表和测试集的索引值的列表
    trainingSet = list(range(50)); testSet = []
    #从50个邮件中，随机挑选出40个作为训练集,10个做测试集
    for i in range(10):
        #随机选取索索引值
        randIndex = int(random.uniform(0, len(trainingSet)))   
        #添加测试集的索引值
        testSet.append(trainingSet[randIndex])      
        #在训练集列表中删除添加到测试集的索引值
        del(trainingSet[randIndex])    
    #创建训练集矩阵和训练集类别标签系向量
    trainMat = []; trainClasses = []               
    #遍历训练集
    for docIndex in trainingSet:  
        #将生成的词集模型添加到训练矩阵中
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))     
        #将类别添加到训练集类别标签系向量中
        trainClasses.append(classList[docIndex])                          
    #训练朴素贝叶斯模型
    p0V, p1V, pSpam = trainNB0(np.array(trainMat), np.array(trainClasses))
    #错误分类计数 
    errorCount = 0                                                        
    #遍历测试集
    for docIndex in testSet:                                              
         #测试集的词集模型
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])        
        #如果分类错误
        if classifyNB(np.array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount += 1                                               
            print("分类错误的测试集：",docList[docIndex])
    print('错误率：%.2f%%' % (float(errorCount) / len(testSet) * 100))

In [48]:
spamTest()